In [1]:
!git clone https://github.com/dmc27/missing_pixels.git

fatal: destination path 'missing_pixels' already exists and is not an empty directory.


In [2]:
from os import listdir
from os.path import isfile

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint

from utils import *

In [3]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Data Preparation

In [4]:
img_path = "datasets"
img_fnames = [f for f in listdir(img_path) if isfile(join(img_path, f))]
print(img_fnames[:20])

['a129163a-1510-489e-b306-f237a3461a33.JPEG', '50da7dac-05ba-4076-8620-31a8d7d730c4.JPEG', '8f2897e9-064b-4196-8e4b-1a331d0d4eb9.JPEG', '6f9b06f7-ef38-4426-9145-0adbd9063a52.JPEG', '1c1989f5-9a3c-4144-863d-cd8c49846057.JPEG', '2f88edb2-59c2-4c89-8719-ba07d95b6b7e.JPEG', 'b54bd8cc-b55c-457d-a479-8caf406b4558.JPEG', '0330d07c-e862-4c64-887a-40e77173edb6.JPEG', '3c8ca092-7a15-4425-9374-9039b1d3deb2.JPEG', '109d6069-67f6-49a7-969b-5b0ece65b149.JPEG', 'f83a5166-b889-4e5c-8065-c1998f8c6438.JPEG', '793a9653-7003-4cfa-93fd-6008d668eb6a.JPEG', '3722a54b-4aeb-4e8c-934a-c9156eb97e11.JPEG', '6244eeef-4970-471c-903a-2f241ecc492e.JPEG', '654d55d9-f3f0-41b4-a8b4-8828638eca46.JPEG', '67adc637-b58c-49c2-9424-93f61a64d8c0.JPEG', '5068b8f5-e7c2-4dc3-9494-0b109d393ea8.JPEG', 'bece99d3-2c47-41a4-b2c3-1ac30806cd77.JPEG', 'b501b44a-ae5c-4884-818d-816f8b0084e3.JPEG', 'a6ab1b64-db5d-4d2f-85e8-26ac2bc99dd0.JPEG']


In [5]:
inputs = []
targets = []
for img_fname in img_fnames:
    img = cv.imread(join(img_path, img_fname), cv.IMREAD_GRAYSCALE)
    img = img.astype(np.float64)

    do_parse_img(img, inputs, targets)

X_train = np.zeros((len(inputs), NUM_INP_NODES))
y_train = np.zeros((len(targets), NUM_OUT_NODES))
for idx in range(len(inputs)):
    X_train[idx, :] = inputs[idx]
    y_train[idx] = targets[idx]

In [6]:
print(X_train.shape)

(7680000, 60)


# Modeling

In [7]:
model_fname = "models/model.mlp.imagenet.h5"

model = Sequential([
    Dense(120, input_shape=(NUM_INP_NODES,), activation="relu"),
    Dense(60, activation="relu"),
    Dense(NUM_OUT_NODES, activation='sigmoid')
])

model.compile(loss="mean_squared_error", optimizer="adam",
              metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               7320      
                                                                 
 dense_1 (Dense)             (None, 60)                7260      
                                                                 
 dense_2 (Dense)             (None, 4)                 244       
                                                                 
Total params: 14824 (57.91 KB)
Trainable params: 14824 (57.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-07-20 13:41:22.942555: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-07-20 13:41:22.942579: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-07-20 13:41:22.942586: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-07-20 13:41:22.942650: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-20 13:41:22.942679: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
checkpoint = ModelCheckpoint(model_fname, monitor='loss', verbose=1,
                             save_best_only=True)
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train, epochs=100, batch_size=512,
                    validation_split=0.1, callbacks=callbacks_list)

Epoch 1/100
    1/13500 [..............................] - ETA: 1:05:54 - loss: 0.0882 - accuracy: 0.2715

2023-07-20 13:41:30.655534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13495/13500 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.3676

2023-07-20 13:42:20.095622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: loss improved from inf to 0.01753, saving model to models/model.mlp.h5
13500/13500 [==============================] - 53s 4ms/step - loss: 0.0175 - accuracy: 0.3676 - val_loss: 0.0150 - val_accuracy: 0.3490
Epoch 2/100
13487/13500 [============================>.] - ETA: 0s - loss: 0.0793 - accuracy: 0.2896
Epoch 2: loss did not improve from 0.01753
13500/13500 [==============================] - 52s 4ms/step - loss: 0.0793 - accuracy: 0.2896 - val_loss: 0.1325 - val_accuracy: 0.2860
Epoch 3/100
13493/13500 [============================>.] - ETA: 0s - loss: 0.1241 - accuracy: 0.2869
Epoch 3: loss did not improve from 0.01753
13500/13500 [==============================] - 51s 4ms/step - loss: 0.1241 - accuracy: 0.2869 - val_loss: 0.1527 - val_accuracy: 0.2963
Epoch 4/100
13499/13500 [============================>.] - ETA: 0s - loss: 0.1186 - accuracy: 0.2909
Epoch 4: loss did not improve from 0.01753
13500/13500 [==============================] - 50s 4ms/step - loss: 0.1186 - ac

In [ ]:
plot_training_history(history)

# Evaluation

In [ ]:
evaluate_mlp(model)